# Weather Image Recognition

# Imports

In [4]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Functions

In [ ]:
def grab_images(directory):
  
  images = []
  
  for root

# Load Dataset

In [3]:
PATH = "/Users/maukanmir/Downloads/dataset/"